In [1]:
import numpy as np

In [17]:
from tdc.multi_pred import DTI
data = DTI(name = 'KIBA')
KibaDataSet = data.get_data()

Found local copy...
Loading...
Done!


In [2]:
import os
import numpy as np

def extract_sequences_from_kiba(kiba_data):
    """
    Extracts protein sequences from the KIBA dataset.
    
    Parameters:
    kiba_data (pd.DataFrame): The KIBA dataset.

    Returns:
    dict: A dictionary where keys are Target_IDs and values are the protein sequences.
    """
    sequence_dict = {}
    for idx, row in kiba_data.iterrows():
        target_id = row['Target_ID']
        sequence = row['Target']
        sequence_dict[target_id] = sequence
    return sequence_dict

def verify_sequence_length_with_contact_map(sequence_dict, contact_map_folder):
    """
    Verifies that the sequence lengths from KIBA match the dimensions of the contact maps.
    
    Parameters:
    sequence_dict (dict): Dictionary containing KIBA protein sequences.
    contact_map_folder (str): Path to the folder containing the contact maps (.npy files).
    
    Returns:
    dict: A dictionary of valid sequences where the sequence length matches the contact map dimensions.
    """
    valid_sequence_dict = {}
    mismatches = []

    for contact_file in os.listdir(contact_map_folder):
        if contact_file.endswith('.npy'):
            protein_id = contact_file.split('.npy')[0]
            contact_map_path = os.path.join(contact_map_folder, contact_file)
            contact_map = np.load(contact_map_path)

            # Check if we have a sequence for this protein
            if protein_id in sequence_dict:
                sequence_length = len(sequence_dict[protein_id])
                contact_map_shape = contact_map.shape[0]

                # Ensure sequence length matches contact map dimensions
                if sequence_length == contact_map_shape:
                    valid_sequence_dict[protein_id] = sequence_dict[protein_id]
                else:
                    mismatches.append(f"Mismatch for {protein_id}: Sequence length = {sequence_length}, Contact map shape = {contact_map_shape}")
            else:
                mismatches.append(f"Target ID {protein_id} not found in KIBA dataset")
    
    return valid_sequence_dict, mismatches

# Example usage
from tdc.multi_pred import DTI
data = DTI(name='KIBA')
kiba_dataset = data.get_data()

# Step 1: Extract sequences from the KIBA dataset
sequence_dict = extract_sequences_from_kiba(kiba_dataset)

# Step 2: Verify that KIBA sequence lengths match the contact map dimensions
contact_map_folder = "./KibaContactMaps"
valid_sequence_dict, mismatches = verify_sequence_length_with_contact_map(sequence_dict, contact_map_folder)

# Output mismatches
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)

# Save the valid sequences to individual text files (if needed)
output_folder = "./KibaValidSequences"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for protein_id, sequence in valid_sequence_dict.items():
    file_path = os.path.join(output_folder, f"{protein_id}.txt")
    with open(file_path, 'w') as f:
        f.write(sequence)

print(f"Valid sequences have been saved to individual text files in {output_folder}")


Found local copy...
Loading...
Done!


Valid sequences have been saved to individual text files in ./KibaValidSequences


In [25]:
# valid_sequence_dict

In [3]:
# import os
# import numpy as np
# import torch
# import torch_geometric.data as DATA
# from torch_geometric.utils import from_networkx
# import networkx as nx

# # Residue properties for feature generation

# res_weight_table = {'A': 71.08, 'C': 103.15, 'D': 115.09, 'E': 129.12, 'F': 147.18, 'G': 57.05, 'H': 137.14,
#                     'I': 113.16, 'K': 128.18, 'L': 113.16, 'M': 131.20, 'N': 114.11, 'P': 97.12, 'Q': 128.13,
#                     'R': 156.19, 'S': 87.08, 'T': 101.11, 'V': 99.13, 'W': 186.22, 'Y': 163.18}

# res_pka_table = {'A': 2.34, 'C': 1.96, 'D': 1.88, 'E': 2.19, 'F': 1.83, 'G': 2.34, 'H': 1.82, 'I': 2.36,
#                  'K': 2.18, 'L': 2.36, 'M': 2.28, 'N': 2.02, 'P': 1.99, 'Q': 2.17, 'R': 2.17, 'S': 2.21,
#                  'T': 2.09, 'V': 2.32, 'W': 2.83, 'Y': 2.32}

# res_pkb_table = {'A': 9.69, 'C': 10.28, 'D': 9.60, 'E': 9.67, 'F': 9.13, 'G': 9.60, 'H': 9.17,
#                  'I': 9.60, 'K': 8.95, 'L': 9.60, 'M': 9.21, 'N': 8.80, 'P': 10.60, 'Q': 9.13,
#                  'R': 9.04, 'S': 9.15, 'T': 9.10, 'V': 9.62, 'W': 9.39, 'Y': 9.62}

# res_pkx_table = {'A': 0.00, 'C': 8.18, 'D': 3.65, 'E': 4.25, 'F': 0.00, 'G': 0, 'H': 6.00,
#                  'I': 0.00, 'K': 10.53, 'L': 0.00, 'M': 0.00, 'N': 0.00, 'P': 0.00, 'Q': 0.00,
#                  'R': 12.48, 'S': 0.00, 'T': 0.00, 'V': 0.00, 'W': 0.00, 'Y': 0.00}

# res_pl_table = {'A': 6.00, 'C': 5.07, 'D': 2.77, 'E': 3.22, 'F': 5.48, 'G': 5.97, 'H': 7.59,
#                 'I': 6.02, 'K': 9.74, 'L': 5.98, 'M': 5.74, 'N': 5.41, 'P': 6.30, 'Q': 5.65,
#                 'R': 10.76, 'S': 5.68, 'T': 5.60, 'V': 5.96, 'W': 5.89, 'Y': 5.96}

# res_hydrophobic_ph2_table = {'A': 47, 'C': 52, 'D': -18, 'E': 8, 'F': 92, 'G': 0, 'H': -42, 'I': 100,
#                              'K': -37, 'L': 100, 'M': 74, 'N': -41, 'P': -46, 'Q': -18, 'R': -26, 'S': -7,
#                              'T': 13, 'V': 79, 'W': 84, 'Y': 49}

# res_hydrophobic_ph7_table = {'A': 41, 'C': 49, 'D': -55, 'E': -31, 'F': 100, 'G': 0, 'H': 8, 'I': 99,
#                              'K': -23, 'L': 97, 'M': 74, 'N': -28, 'P': -46, 'Q': -10, 'R': -14, 'S': -5,
#                              'T': 13, 'V': 76, 'W': 97, 'Y': 63}

# # Residue group properties
# aliphatic = ['A', 'I', 'L', 'M', 'V']
# aromatic = ['F', 'W', 'Y']
# polar_neutral = ['C', 'N', 'Q', 'S', 'T']
# acidic_charged = ['D', 'E']
# basic_charged = ['H', 'K', 'R']

# # 1. Function to generate the full node feature vector for each residue

# def residue_features(residue):
#     """
#     Generates a feature vector for each residue (amino acid).
    
#     Parameters:
#     residue (str): The single-letter amino acid code (e.g., 'A', 'C').

#     Returns:
#     numpy array: A 54-dimensional feature vector for the residue.
#     """
#     # One-hot encoding of the residue (21-dimensional)
#     one_hot = [1 if residue == r else 0 for r in res_weight_table.keys()]

#     # Residue group properties (binary flags)
#     aliphatic_flag = [1] if residue in aliphatic else [0]
#     aromatic_flag = [1] if residue in aromatic else [0]
#     polar_neutral_flag = [1] if residue in polar_neutral else [0]
#     acidic_charged_flag = [1] if residue in acidic_charged else [0]
#     basic_charged_flag = [1] if residue in basic_charged else [0]

#     # Chemical properties (single scalar values)
#     weight = [res_weight_table.get(residue, 0)]  # Molecular weight
#     pka = [res_pka_table.get(residue, 0)]  # pKa of -COOH
#     pkb = [res_pkb_table.get(residue, 0)]  # pKb of -NH3
#     pkx = [res_pkx_table.get(residue, 0)]  # pKa of side chain
#     pi = [res_pl_table.get(residue, 0)]  # Isoelectric point (pI)
#     hydrophobicity_ph2 = [res_hydrophobic_ph2_table.get(residue, 0)]  # Hydrophobicity at pH 2
#     hydrophobicity_ph7 = [res_hydrophobic_ph7_table.get(residue, 0)]  # Hydrophobicity at pH 7

#     # Combine all features into a single vector (54-dimensional)
#     feature_vector = (
#         one_hot +
#         aliphatic_flag +
#         aromatic_flag +
#         polar_neutral_flag +
#         acidic_charged_flag +
#         basic_charged_flag +
#         weight +
#         pka +
#         pkb +
#         pkx +
#         pi +
#         hydrophobicity_ph2 +
#         hydrophobicity_ph7
#     )
    
#     return np.array(feature_vector)

# # 2. Function to create the graph from the contact map and node features

# def create_graph_from_contact_map(protein_sequence, contact_map):
#     """
#     Creates a graph from a protein contact map, adding node features based on residue properties.

#     Parameters:
#     protein_sequence (str): The amino acid sequence of the protein.
#     contact_map (numpy array): The NxN contact map matrix where each entry indicates a contact.

#     Returns:
#     torch_geometric.data.Data: The graph data in PyTorch Geometric format.
#     """
#     num_residues = len(protein_sequence)  # Number of residues in the sequence

#     # Create a NetworkX graph
#     G = nx.Graph()

#     # Add nodes with residue features
#     for i, residue in enumerate(protein_sequence):
#         G.add_node(i, features=residue_features(residue))

#     # Add edges based on the contact map (threshold: contact_map >= 0.5)
#     for i in range(num_residues):
#         for j in range(i + 1, num_residues):
#             if contact_map[i, j] >= 0.5:  # Threshold for contact :todo: chnage it nd tune it for better results !!
#                 G.add_edge(i, j)

#     # Convert the NetworkX graph to PyTorch Geometric format
#     data = from_networkx(G)

#     # Extract node features into the `x` attribute (used by PyTorch Geometric)
#     data.x = torch.tensor([G.nodes[i]['features'] for i in G.nodes], dtype=torch.float)

#     return data

# # 3. Process the valid sequences and corresponding contact maps

# def process_proteins(sequence_dict, contact_map_folder):
#     """
#     Processes all proteins: loads contact maps and creates graphs with node features for each protein.

#     Parameters:
#     sequence_dict (dict): Dictionary containing the valid protein sequences.
#     contact_map_folder (str): Path to the folder containing the contact maps.

#     Returns:
#     dict: A dictionary where keys are protein IDs and values are graph data objects.
#     """
#     graph_data_dict = {}

#     for contact_file in os.listdir(contact_map_folder):
#         if contact_file.endswith('.npy'):
#             protein_id = contact_file.split('.npy')[0]
            
#             if protein_id in sequence_dict:
#                 # Load the contact map
#                 contact_map_path = os.path.join(contact_map_folder, contact_file)
#                 contact_map = np.load(contact_map_path)

#                 # Retrieve the protein sequence
#                 sequence = sequence_dict[protein_id]

#                 # Create the graph from the contact map and sequence
#                 graph_data = create_graph_from_contact_map(sequence, contact_map)
                
#                 # Store the graph data in the dictionary
#                 graph_data_dict[protein_id] = graph_data
#             else:
#                 print(f"Warning: No sequence found for protein {protein_id}")

#     return graph_data_dict

# # Example usage

# contact_map_folder = "./KibaContactMaps"

# # Assuming valid_sequence_dict is already populated
# graph_data_dict = process_proteins(valid_sequence_dict, contact_map_folder)

# # You can now use graph_data_dict for downstream tasks
# for protein_id, graph in graph_data_dict.items():
#     print(f"Graph for {protein_id}: {graph}")


\section*{Position-Specific Scoring Matrix (PSSM)}

A **Position-Specific Scoring Matrix (PSSM)** is a matrix used in bioinformatics to represent the likelihood of finding a particular amino acid at each position of a protein sequence, based on an alignment of multiple sequences. The matrix is derived from a **multiple sequence alignment (MSA)** of related protein sequences, where each column represents the probabilities of different amino acids appearing at a specific position.

Each row in the matrix corresponds to a position in the sequence (i.e., one residue in the protein), and each column corresponds to one of the 20 standard amino acids (with a possible additional column for unknown residues or gaps).

\subsection*{Simple Example}

Imagine we have the following **protein sequence**:

\[
\text{Sequence}: A T G C
\]

We also have several **aligned sequences** from other similar proteins:

\[
\begin{aligned}
&\text{A T G C} \\
&\text{A A G C} \\
&\text{T T G C} \\
&\text{A T T C} \\
\end{aligned}
\]

Here, we have four sequences of length 4. For each position in the protein sequence, we count how often each amino acid appears.

For example, at the **first position**:
- 'A' appears 3 times.
- 'T' appears 1 time.

At the **second position**:
- 'T' appears 3 times.
- 'A' appears 1 time.

\subsection*{Position Frequency Matrix (PFM)}

The **Position Frequency Matrix (PFM)** is the raw count of amino acid occurrences in each position of the alignment. For the example sequence \texttt{A T G C}, the PFM looks like:

\[
\begin{array}{c|c|c|c|c}
\text{Position} & A & T & G & C \\
\hline
1 & 3 & 1 & 0 & 0 \\
2 & 1 & 3 & 0 & 0 \\
3 & 0 & 0 & 4 & 0 \\
4 & 0 & 0 & 0 & 4 \\
\end{array}
\]

\subsection*{Adding Pseudocounts}

To avoid zero probabilities, we add **pseudocounts** to the matrix. A small value (e.g., 0.8) is added to each count to ensure no entry in the matrix is zero.

\subsection*{Position-Specific Scoring Matrix (PSSM)}

To calculate the **PSSM**, we normalize the frequencies by dividing the counts by the total number of sequences (in this case, 4):

\[
\begin{array}{c|c|c|c|c}
\text{Position} & A & T & G & C \\
\hline
1 & 0.75 & 0.25 & 0.00 & 0.00 \\
2 & 0.25 & 0.75 & 0.00 & 0.00 \\
3 & 0.00 & 0.00 & 1.00 & 0.00 \\
4 & 0.00 & 0.00 & 0.00 & 1.00 \\
\end{array}
\]

This table shows the likelihood of each amino acid appearing at each position. The PSSM is essentially this matrix of probabilities (though in advanced cases, log-odds values may be used).

\subsection*{How We Calculate PSSM in the Code}

In the code, the **PSSM calculation** is performed as follows:
\begin{enumerate}
    \item **Reading an alignment file**: The multiple sequence alignment (MSA) is read from an alignment file.
    \item **Counting occurrences**: For each position in the alignment, the number of occurrences of each amino acid is counted (creating the PFM).
    \item **Adding pseudocounts**: Small pseudocounts are added to avoid zero probabilities.
    \item **Normalizing**: The counts are divided by the total number of sequences to create a probability matrix (PPM), which is treated as the PSSM.
\end{enumerate}

\subsection*{Conclusion}

The PSSM matrix provides a way to capture the variability of amino acids at specific positions within an alignment, and it is widely used in various bioinformatics applications such as sequence alignment and protein structure prediction.


In [ ]:
import os
import numpy as np
import torch
import torch_geometric.data as DATA
from torch_geometric.utils import from_networkx
import networkx as nx

# Residue properties for feature generation
res_weight_table = {'A': 71.08, 'C': 103.15, 'D': 115.09, 'E': 129.12, 'F': 147.18, 'G': 57.05, 'H': 137.14,
                    'I': 113.16, 'K': 128.18, 'L': 113.16, 'M': 131.20, 'N': 114.11, 'P': 97.12, 'Q': 128.13,
                    'R': 156.19, 'S': 87.08, 'T': 101.11, 'V': 99.13, 'W': 186.22, 'Y': 163.18}

res_pka_table = {'A': 2.34, 'C': 1.96, 'D': 1.88, 'E': 2.19, 'F': 1.83, 'G': 2.34, 'H': 1.82, 'I': 2.36,
                 'K': 2.18, 'L': 2.36, 'M': 2.28, 'N': 2.02, 'P': 1.99, 'Q': 2.17, 'R': 2.17, 'S': 2.21,
                 'T': 2.09, 'V': 2.32, 'W': 2.83, 'Y': 2.32}

res_pkb_table = {'A': 9.69, 'C': 10.28, 'D': 9.60, 'E': 9.67, 'F': 9.13, 'G': 9.60, 'H': 9.17,
                 'I': 9.60, 'K': 8.95, 'L': 9.60, 'M': 9.21, 'N': 8.80, 'P': 10.60, 'Q': 9.13,
                 'R': 9.04, 'S': 9.15, 'T': 9.10, 'V': 9.62, 'W': 9.39, 'Y': 9.62}

res_pkx_table = {'A': 0.00, 'C': 8.18, 'D': 3.65, 'E': 4.25, 'F': 0.00, 'G': 0, 'H': 6.00,
                 'I': 0.00, 'K': 10.53, 'L': 0.00, 'M': 0.00, 'N': 0.00, 'P': 0.00, 'Q': 0.00,
                 'R': 12.48, 'S': 0.00, 'T': 0.00, 'V': 0.00, 'W': 0.00, 'Y': 0.00}

res_pl_table = {'A': 6.00, 'C': 5.07, 'D': 2.77, 'E': 3.22, 'F': 5.48, 'G': 5.97, 'H': 7.59,
                'I': 6.02, 'K': 9.74, 'L': 5.98, 'M': 5.74, 'N': 5.41, 'P': 6.30, 'Q': 5.65,
                'R': 10.76, 'S': 5.68, 'T': 5.60, 'V': 5.96, 'W': 5.89, 'Y': 5.96}

res_hydrophobic_ph2_table = {'A': 47, 'C': 52, 'D': -18, 'E': 8, 'F': 92, 'G': 0, 'H': -42, 'I': 100,
                             'K': -37, 'L': 100, 'M': 74, 'N': -41, 'P': -46, 'Q': -18, 'R': -26, 'S': -7,
                             'T': 13, 'V': 79, 'W': 84, 'Y': 49}

res_hydrophobic_ph7_table = {'A': 41, 'C': 49, 'D': -55, 'E': -31, 'F': 100, 'G': 0, 'H': 8, 'I': 99,
                             'K': -23, 'L': 97, 'M': 74, 'N': -28, 'P': -46, 'Q': -10, 'R': -14, 'S': -5,
                             'T': 13, 'V': 76, 'W': 97, 'Y': 63}

# Residue group properties
aliphatic = ['A', 'I', 'L', 'M', 'V']
aromatic = ['F', 'W', 'Y']
polar_neutral = ['C', 'N', 'Q', 'S', 'T']
acidic_charged = ['D', 'E']
basic_charged = ['H', 'K', 'R']

# Function to calculate PSSM from an alignment file and the protein sequence
def PSSM_calculation(aln_file, pro_seq):
    """
    Calculates the PSSM matrix from the alignment file and protein sequence, ignoring gaps in alignment.
    
    Parameters:
    aln_file (str): Path to the alignment file (.aln)
    pro_seq (str): The corresponding protein sequence
    
    Returns:
    numpy array: PSSM matrix (Nx21 where N is the length of the protein sequence)
    """
    pfm_mat = np.zeros((len(res_weight_table), len(pro_seq)))  # Frequency matrix initialization
    with open(aln_file, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]  # Read non-empty lines
        line_count = len(lines)  # Number of sequences

        for line in lines:
            if len(line) != len(pro_seq):  # Ignore sequences that don't match the expected length
                print(f"Skipping line with mismatched length: {len(line)} vs expected {len(pro_seq)}")
                continue
            count = 0
            for res in line:
                if res in res_weight_table:
                    pfm_mat[list(res_weight_table.keys()).index(res), count] += 1
                count += 1

    # Convert PFM to PPM using pseudocounts
    pseudocount = 0.8
    ppm_mat = (pfm_mat + pseudocount / len(res_weight_table)) / (line_count + pseudocount)

    # Ensure PSSM matrix has correct dimensions (length of sequence, 21 features)
    if ppm_mat.shape[0] != len(pro_seq):
        ppm_mat = ppm_mat.T  # Transpose if necessary

    return ppm_mat  # The PPM is used as the PSSM matrix in this case



# Function to generate the full node feature vector for each residue
def residue_features(residue, pssm_vector):
    """
    Generates a feature vector for each residue (amino acid), including PSSM and other biochemical properties.

    Parameters:
    residue (str): The single-letter amino acid code (e.g., 'A', 'C').
    pssm_vector (numpy array): The PSSM vector for this residue (21-dimensional).

    Returns:
    numpy array: A 54-dimensional feature vector for the residue.
    """
    # One-hot encoding of the residue (21-dimensional)
    one_hot = [1 if residue == r else 0 for r in res_weight_table.keys()]

    # Residue group properties (binary flags)
    aliphatic_flag = [1] if residue in aliphatic else [0]
    aromatic_flag = [1] if residue in aromatic else [0]
    polar_neutral_flag = [1] if residue in polar_neutral else [0]
    acidic_charged_flag = [1] if residue in acidic_charged else [0]
    basic_charged_flag = [1] if residue in basic_charged else [0]

    # Chemical properties (single scalar values)
    weight = [res_weight_table.get(residue, 0)]  # Molecular weight
    pka = [res_pka_table.get(residue, 0)]  # pKa of -COOH
    pkb = [res_pkb_table.get(residue, 0)]  # pKb of -NH3
    pkx = [res_pkx_table.get(residue, 0)]  # pKa of side chain
    pi = [res_pl_table.get(residue, 0)]  # Isoelectric point (pI)
    hydrophobicity_ph2 = [res_hydrophobic_ph2_table.get(residue, 0)]  # Hydrophobicity at pH 2
    hydrophobicity_ph7 = [res_hydrophobic_ph7_table.get(residue, 0)]  # Hydrophobicity at pH 7

    # Combine all features into a single vector (54-dimensional)
    feature_vector = (
        one_hot +
        list(pssm_vector) +  # PSSM vector (21-dimensional)
        aliphatic_flag +
        aromatic_flag +
        polar_neutral_flag +
        acidic_charged_flag +
        basic_charged_flag +
        weight +
        pka +
        pkb +
        pkx +
        pi +
        hydrophobicity_ph2 +
        hydrophobicity_ph7
    )
    
    return np.array(feature_vector)

# Function to create the graph from the contact map and node features
def create_graph_from_contact_map(protein_sequence, pssm_matrix, contact_map):
    """
    Creates a graph from a protein contact map, adding node features based on residue properties.

    Parameters:
    protein_sequence (str): The amino acid sequence of the protein.
    pssm_matrix (numpy array): The PSSM matrix for the protein (Nx21 for N residues).
    contact_map (numpy array): The NxN contact map matrix where each entry indicates a contact.

    Returns:
    torch_geometric.data.Data: The graph data in PyTorch Geometric format.
    """
    num_residues = len(protein_sequence)  # Number of residues in the sequence

    # Ensure PSSM matrix matches the sequence length
    if len(pssm_matrix) != num_residues:
        print(pssm_matrix)
        print(pssm_matrix.shape)
        raise ValueError(f"Mismatch between sequence length ({num_residues}) and PSSM matrix size ({len(pssm_matrix)}).")

    # Create a NetworkX graph
    G = nx.Graph()

    # Add nodes with residue features
    for i, residue in enumerate(protein_sequence):
        pssm_vector = pssm_matrix[i]  # Get the PSSM vector for residue i
        G.add_node(i, features=residue_features(residue, pssm_vector))

    # Add edges based on the contact map (threshold: contact_map >= 0.5)
    for i in range(num_residues):
        for j in range(i + 1, num_residues):
            if contact_map[i, j] >= 0.5:  # Threshold for contact
                G.add_edge(i, j)

    # Convert the NetworkX graph to PyTorch Geometric format
    data = from_networkx(G)

    # Extract node features into the `x` attribute (used by PyTorch Geometric)
    data.x = torch.tensor([G.nodes[i]['features'] for i in G.nodes], dtype=torch.float)

    return data


# Process the valid sequences and corresponding contact maps
def process_proteins(sequence_dict, contact_map_folder, aln_folder, save_folder):
    """
    Processes all proteins: loads contact maps, calculates PSSM matrices, creates graphs with node features, 
    and saves the graphs to files.
    A counter tracks how many graphs are created and how many are left.
    
    Parameters:
    sequence_dict (dict): Dictionary containing the valid protein sequences.
    contact_map_folder (str): Path to the folder containing the contact maps.
    aln_folder (str): Path to the folder containing the alignment files (.aln).
    save_folder (str): Path to the folder where the graph files will be saved.
    
    Returns:
    dict: A dictionary where keys are protein IDs and values are graph data objects.
    """
    graph_data_dict = {}

    # Ensure the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Get the total number of .npy files (contact maps) in the folder
    contact_files = [f for f in os.listdir(contact_map_folder) if f.endswith('.npy')]
    total_files = len(contact_files)
    completed = 0

    # Process each contact map
    for contact_file in contact_files:
        protein_id = contact_file.split('.npy')[0]

        if protein_id in sequence_dict:
            # Load the contact map
            contact_map_path = os.path.join(contact_map_folder, contact_file)
            contact_map = np.load(contact_map_path)

            # Retrieve the protein sequence
            sequence = sequence_dict[protein_id]

            # Calculate the PSSM matrix for this sequence
            aln_file = os.path.join(aln_folder, f"{protein_id}.aln")
            pssm_matrix = PSSM_calculation(aln_file, sequence)

            # Create the graph from the contact map and sequence
            graph_data = create_graph_from_contact_map(sequence, pssm_matrix, contact_map)

            # Store the graph data in the dictionary
            graph_data_dict[protein_id] = graph_data

            # Save the graph to a file
            graph_save_path = os.path.join(save_folder, f"{protein_id}_graph.pt")
            torch.save(graph_data, graph_save_path)

            # Update the counter
            completed += 1
            print(f"Created and saved graph {completed}/{total_files}. Remaining: {total_files - completed}")
        else:
            print(f"Warning: No sequence found for protein {protein_id}")

    return graph_data_dict
# Example usage
contact_map_folder = "./KibaContactMaps"
aln_folder = "./data/kiba/aln"
save_folder = "./ProteinGraphs"

# Assuming valid_sequence_dict is already populated
graph_data_dict = process_proteins(valid_sequence_dict, contact_map_folder, aln_folder,save_folder)

# You can now use graph_data_dict for downstream tasks
for protein_id, graph in graph_data_dict.items():
    print(f"Graph for {protein_id}: {graph}")


Created and saved graph 1/203. Remaining: 202
Created and saved graph 2/203. Remaining: 201
Created and saved graph 3/203. Remaining: 200
Created and saved graph 4/203. Remaining: 199
Created and saved graph 5/203. Remaining: 198
